# Cкачивание текстов для проекта Крюков-Шолохов

## Prepare service data

In [75]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import re
import os

req_headers={'accept': 'text/plain, text/html',
'accept-encoding': 'gzip, deflate, br',
'accept-language': 'ru-RU,ru;q=0.9,en-US;q=0.8,en;q=0.7',
'cache-control': 'max-age=0',
'user-agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Ubuntu Chromium/69.0.3497.81 Chrome/69.0.3497.81 Safari/537.36'}

def text_clear(strText, TypeOfClear=1):
    t=strText
    if TypeOfClear==1:
        t=re.sub('\n+', ' ', t)
        t=re.sub('<BR>', '\n', t)
        t=re.sub('\n +', '\n', t)

    t=re.sub('—', '-', t)
    t=re.sub('-{3,}', '', t)
    t=re.sub('--', '-', t)
    t=re.sub(' +', ' ', t)
    return t.strip()

def iterate_group(iterator, count):
    itr=iter(iterator)
    for i in range(0, len(iterator), count):
        yield iterator[i:i+count]

In [2]:
%run main_serv.py
auth_load=authors.copy()

In [3]:
auth_load.use(val=False)
auth_load.use(index=['sera', 'fad', 'bab', 'kat', 'pla']) # load these authors

not found param key_word "index" so set value to ALL authors


In [4]:
auth_load

,file_mask,fire,in_use,strFileDir,strRuName,strUrl
index,,,,,,
kru,*,0,False,Krukov,Ф. Крюков,http://az.lib.ru/k/krjukow_f_d/
sho,*.txt,0,False,Sholokhov,М. Шолохов,http://www.lib.ru/PROZA/SHOLOHOW/
ltol,*,0,False,Tolstoy,Л. Толстой,
sera,*,0,True,Serafimovich,А. Серафимович,http://lib.ru/RUSSLIT/SERAFIMOWICH/
fad,*,0,True,Fadeyev,А. Фадеев,http://www.lib.ru/RUSSLIT/FADEEW/
bab,*,0,True,Babel,И. Бабель,http://www.lib.ru/PROZA/BABEL/
kat,*,0,True,Kataev,В. Катаев,http://www.lib.ru/PROZA/KATAEW/
pla,*,0,True,Platonov,А. Платонов,http://platonov-ap.ru/novels/


### Serafimovich

In [6]:
strSERABase=auth_load.loc['sera', 'strUrl']

def read_sera(strSERAURL=strSERABase):
    def work_with_item(strURL, name, sess):
        ser_t=sess.get(strSERABase+strURL)
        ser_t.encoding=ser_t.apparent_encoding
        sp_t=BeautifulSoup(ser_t.text, 'html.parser')
    
        mo=re.search(r'(\n?-{4,} ?\n)', sp_t.text)
        text=sp_t.text[mo.start(1)+mo.end(1):]
    
        text=re.sub('\n  ', '<BR>', text)
        text=re.split('\n?\d+\n', text)
    
        text=list(map(text_clear, text))
    
        for n, t in enumerate(text[:-1]):
            strFileName=os.path.join(auth_load.loc['sera', 'strFileDir'], '{name}_{num}.txt'.format(name=name, num=n+1))
            print('work with ', strFileName, end=' ')
        
            with open(strFileName, 'w') as flw:
                flw.write(t)
                print('Done', end='\n')
        return text

    with requests.session() as sess:
        sess.headers.update(req_headers)
        ser_m=sess.get(strSERAURL)
        ser_m.encoding=ser_m.apparent_encoding
        sp_ser=BeautifulSoup(ser_m.text, 'html.parser')

        links=sp_ser.findAll(lambda x: x.name == 'a' and re.search(r'\.txt\b', x['href']))

        try:
            os.mkdir(auth_load.loc['sera', 'strFileDir'])
        except:
            pass
    
        for link in links:
            work_with_item(link['href'], link.text, sess)

        print('All done.')
        
#read_sera()

### Fadeyev

In [15]:
strFADURL=auth_load.loc['fad', 'strUrl']

with requests.session() as sess:
    sess.headers.update(req_headers)
    fad_m=sess.get(strFADURL)
    fad_m.encoding=fad_m.apparent_encoding
    sp_fad=BeautifulSoup(fad_m.text, 'html.parser')

    #links=sp_fad.findAll(lambda x: x.name == 'a' and re.search(r'\.txt\b', x.get('href')))
    links=[l for l in sp_fad.findAll('a', href=True) if re.search(r'\.txt\b', l['href'])]

    try:
        os.mkdir(auth_load.loc['fad', 'strFileDir'])
    except:
        pass
    
    for link in links:
        print(link['href'], link.text)

    print('All done.')

razgrom.txt Разгром
mol_gwardiya.txt Молодая гвардия
amgun.txt Рождение амгуньского полка
bednbogt.txt О бедности и богатстве
metal.txt Черная металлургия
odin.txt Один в чаще
razliv.txt Разлив
udege.txt Последний из удэге
zemletr.txt Землетрясение
ofadeeve.txt В.Озеров. Певец революционной эпохи
All done.


In [16]:
s=sess.get(strFADURL+links[0]['href'])

s.encoding=s.apparent_encoding
sp=BeautifulSoup(s.text, 'html.parser')

text_html=re.split('<ul><a name=\d+></a><h2>[IVXА-Яа-я., ]+</h2></ul>', s.text)
text=[t.strip() for t in text_html if not (re.search(r'(\n?-{4,} ?\n)', t) or re.search(r'<form action=', t))]

for i in text:
    print(i[:250])

Бренча по ступенькам избитой японской шашкой, Левинсон вышел во двор. С
полей тянуло гречишным медом. В жаркой бело-розовой пене плавало над головой
июльское солнце.
     Ординарец  Морозка,  отгоняя  плетью  осатаневших   цесарок,  сушил  на
брезент
Сказать правду, спасенный не понравился Морозке с первого взгляда.
     Морозка не  любил чистеньких людей.  В его  жизненной практике это были
непостоянные, никчемные люди,  которым нельзя верить. Кроме того,  раненый с
первых же шагов проявил себя 
Морозка и Варя вернулись за полдень, не глядя друг на  друга, усталые и
ленивые.
     Морозка вышел на  прогалину и, заложив  два пальца в  рот, свистнул три
раза пронзительным разбойным свистом. И когда, как в сказке, вылетел из чащи
курчавый, звонк
Приезд Морозки нарушил душевное равновесие, установившееся в Мечике под
влиянием ровной, безмятежной жизни в госпитале.
     "Почему  он  смотрел  так  пренебрежительно? -- подумал  Ме-чик,  когда
ординарец уехал. --  Пусть он вытащил меня из  огн

In [18]:
def work_with_item(strURL, name, sess):

    s=sess.get(strURL)

    s.encoding=s.apparent_encoding
    sp=BeautifulSoup(s.text, 'html.parser')

    text_html=re.sub('\n  ', '<BR>', s.text)
    
    text_html=re.split('<ul><a name=\d+></a><h2>[0-9IVXА-Яа-я?!*., ]+</h2></ul>', text_html)
    
    text=[t.strip() for t in text_html if not (re.search(r'(\n?-{4,} ?\n)', t) or re.search(r'<form action=', t))]

    text=list(map(text_clear, text))
    
    for n, t in enumerate(text):
        #strFileName=os.path.join(auth_load.loc['fad', 'strFileDir'], '{name}_{num}.txt'.format(name=name, num=n+1))
        strFileName=os.path.join('prep', '{name}_{num}.txt'.format(name=name, num=n+1))
        print('work with ', strFileName, end=' ')
        
        with open(strFileName, 'w') as flw:
            flw.write(t)
            print('Done', end='\n')


In [21]:
ind=9
for i in links:
    work_with_item(strFADURL+i['href'], i.text, sess)

work with  prep/Разгром_1.txt Done
work with  prep/Разгром_2.txt Done
work with  prep/Разгром_3.txt Done
work with  prep/Разгром_4.txt Done
work with  prep/Разгром_5.txt Done
work with  prep/Разгром_6.txt Done
work with  prep/Разгром_7.txt Done
work with  prep/Разгром_8.txt Done
work with  prep/Разгром_9.txt Done
work with  prep/Разгром_10.txt Done
work with  prep/Разгром_11.txt Done
work with  prep/Разгром_12.txt Done
work with  prep/Разгром_13.txt Done
work with  prep/Разгром_14.txt Done
work with  prep/Разгром_15.txt Done
work with  prep/Молодая гвардия_1.txt Done
work with  prep/Молодая гвардия_2.txt Done
work with  prep/Молодая гвардия_3.txt Done
work with  prep/Молодая гвардия_4.txt Done
work with  prep/Рождение амгуньского полка_1.txt Done
work with  prep/Рождение амгуньского полка_2.txt Done
work with  prep/Рождение амгуньского полка_3.txt Done
work with  prep/Рождение амгуньского полка_4.txt Done
work with  prep/Рождение амгуньского полка_5.txt Done
work with  prep/Рождение ам

work with  prep/Землетрясение_1.txt Done
work with  prep/Землетрясение_2.txt Done
work with  prep/Землетрясение_3.txt Done
work with  prep/Землетрясение_4.txt Done
work with  prep/Землетрясение_5.txt Done
work with  prep/Землетрясение_6.txt Done
work with  prep/В.Озеров. Певец революционной эпохи_1.txt Done
work with  prep/В.Озеров. Певец революционной эпохи_2.txt Done
work with  prep/В.Озеров. Певец революционной эпохи_3.txt Done
work with  prep/В.Озеров. Певец революционной эпохи_4.txt Done


### Platonov

In [22]:
strPLAURL=auth_load.loc['pla', 'strUrl']

with requests.session() as sess:
    sess.headers.update(req_headers)
    pla_m=sess.get(strPLAURL)
    pla_m.encoding=pla_m.apparent_encoding
    sp_pla=BeautifulSoup(pla_m.text, 'html.parser')

    #links=sp_fad.findAll(lambda x: x.name == 'a' and re.search(r'\.txt\b', x.get('href')))
    links=[l for l in sp_pla.findAll('a', href=True) if re.search(r'http://platonov-ap.ru/novels/', l['href'])]

    try:
        os.mkdir(auth_load.loc['pla', 'strFileDir'])
    except:
        pass
    
    for link in links:
        print(link['href'], link.text)

    print('All done.')

http://platonov-ap.ru/novels/v-okrestnostyah-odnoy-turmy/ «В окрестностях одной тюрьмы…»1927
http://platonov-ap.ru/novels/pervyi-ivan/ «Первый Иван». Заметки о техническом творчестве трудящихся людей1930
http://platonov-ap.ru/novels/chelust-drakona/ «Челюсть дракона»1945
http://platonov-ap.ru/novels/doklad-upravleniya-rabot-po-gidrofikacii/ <Доклад управления работ по гидрофикации центральной Азии>1922
http://platonov-ap.ru/novels/alterke/ Алтеркэ1940
http://platonov-ap.ru/novels/antisexus/ Антисексус1926
http://platonov-ap.ru/novels/apalitych/ Апалитыч1920
http://platonov-ap.ru/novels/afrodita/ Афродита1944
http://platonov-ap.ru/novels/belogorlik/ Белогорлик1920
http://platonov-ap.ru/novels/bessmertie/ Бессмертие1936
http://platonov-ap.ru/novels/bozhie-derevo/ Божье дерево1943
http://platonov-ap.ru/novels/boi-v-grozu/ Бой в грозу1943
http://platonov-ap.ru/novels/bronya/ Броня1942
http://platonov-ap.ru/novels/buchilo/ Бучило1924
http://platonov-ap.ru/novels/v-zvezdnoy-pustyne/ В звездн

In [69]:
def work_with_item_pla(strURL, name, sess):

    s=sess.get(strURL)
    
    s.encoding=s.apparent_encoding
    sp=BeautifulSoup(s.text, 'html.parser')
    art=sp.find('article')
    #print(art)
    ps=art.findAll('p')
    h1=art.find('h1')
    strFileName=os.path.join('Platonov', '{name}.txt'.format(name=h1.text))
    
    text=''
    if len(ps)>0:
        text='\n'.join([p.text for p in ps])
    else:
        text=art.text
    
    print('work with ', strFileName, end=' ')
    with open(strFileName, 'w') as flw:
        flw.write(text_clear(text, TypeOfClear=2))
        print('Done', end='\n')
            
for link in links:
    work_with_item_pla(link['href'], '', sess)
     #print(link['href'], re.sub('\d+', '', link.text))

work with  Platonov/«В окрестностях одной тюрьмы…».txt Done
work with  Platonov/«Первый Иван». Заметки о техническом творчестве трудящихся людей.txt Done
work with  Platonov/«Челюсть дракона».txt Done
work with  Platonov/<Доклад управления работ по гидрофикации центральной Азии>.txt Done
work with  Platonov/Алтеркэ.txt Done
work with  Platonov/Антисексус.txt Done
work with  Platonov/Апалитыч.txt Done
work with  Platonov/Афродита.txt Done
work with  Platonov/Белогорлик.txt Done
work with  Platonov/Бессмертие.txt Done
work with  Platonov/Божье дерево.txt Done
work with  Platonov/Бой в грозу.txt Done
work with  Platonov/Броня.txt Done
work with  Platonov/Бучило.txt Done
work with  Platonov/В звездной пустыне.txt Done
work with  Platonov/В мастерских.txt Done
work with  Platonov/В прекрасном и яростном мире (Машинист Мальцев).txt Done
work with  Platonov/Великий человек.txt Done
work with  Platonov/Верное сердце.txt Done
work with  Platonov/Взыскание погибших.txt Done
work with  Platonov/В

### Chevengur

In [94]:
def read_che():
    strChe_URL=r'http://platonov-ap.ru/romans/chevengur/'

    s=sess.get(strChe_URL)
  
    s.encoding=s.apparent_encoding
    sp=BeautifulSoup(s.text, 'html.parser')
    art=sp.find('article')
    h1=art.find('h1')

    ps=art.findAll('p')
    h1=art.find('h1')
    
    ps=[text_clear(p.text.replace(u'\xa0', u' '), TypeOfClear=2) for p in ps]
    txtl=[t for t in iterate_group(ps, 50)]
    for n, t in enumerate(txtl):
        strFileName=os.path.join('predict/pl1', '{name}_{num}.txt'.format(name=h1.text, num=n))
        print('work with ', strFileName, end=' ')
        text='\n'.join([p for p in t])
        
        with open(strFileName, 'w') as flw:
            flw.write(text_clear(text, TypeOfClear=2))
            print('Done', end='\n')

    #print('work with ', strFileName, end=' ')
    
read_che()

work with  predict/pl1/Чевенгур_0.txt Done
work with  predict/pl1/Чевенгур_1.txt Done
work with  predict/pl1/Чевенгур_2.txt Done
work with  predict/pl1/Чевенгур_3.txt Done
work with  predict/pl1/Чевенгур_4.txt Done
work with  predict/pl1/Чевенгур_5.txt Done
work with  predict/pl1/Чевенгур_6.txt Done
work with  predict/pl1/Чевенгур_7.txt Done
work with  predict/pl1/Чевенгур_8.txt Done
work with  predict/pl1/Чевенгур_9.txt Done
work with  predict/pl1/Чевенгур_10.txt Done
work with  predict/pl1/Чевенгур_11.txt Done
work with  predict/pl1/Чевенгур_12.txt Done
work with  predict/pl1/Чевенгур_13.txt Done
work with  predict/pl1/Чевенгур_14.txt Done
work with  predict/pl1/Чевенгур_15.txt Done
work with  predict/pl1/Чевенгур_16.txt Done


## Kataev

In [135]:
strFile1='02_-_Hutorok_v_stepi.txt'
strFile2='03_-_Zimniy_Veter.txt'
strFile3='Beleet_parus_odinokiy.txt'
def get_text(strFile=strFile1, strSplitTemplate=r'(?m)\d+\. [А-Яа-я«» ]+\n', 
             name='хуторок_в_степи', write_file=True):
    strPREP=r'prep/kataev'
    strFileName=os.path.join(strPREP, strFile)

    with open(strFileName, 'r') as flr:
        text=flr.read()
        #mo=re.findall(r'(?m)\d+\. [А-Яа-я«» ]+\n', text)
        text=re.split(strSplitTemplate, text)
        
        for n, t in enumerate(text):
            
            if write_file:
                strFileName=os.path.join(strPREP, '{name}_{num}.txt'.format(name=name, num=n))
                print('work with ', strFileName, end=' ')
        
                with open(strFileName, 'w') as flw:
                    flw.write(text_clear(t, TypeOfClear=2))
                    print('Done', end='\n')
            else:
                print(n, t[:100])
                
get_text(strFile=strFile3, strSplitTemplate=r'(?m)\d+ [А-Я«»!…, ]+\n', name='белеет_парус')

work with  prep/kataev/белеет_парус_0.txt Done
work with  prep/kataev/белеет_парус_1.txt Done
work with  prep/kataev/белеет_парус_2.txt Done
work with  prep/kataev/белеет_парус_3.txt Done
work with  prep/kataev/белеет_парус_4.txt Done
work with  prep/kataev/белеет_парус_5.txt Done
work with  prep/kataev/белеет_парус_6.txt Done
work with  prep/kataev/белеет_парус_7.txt Done
work with  prep/kataev/белеет_парус_8.txt Done
work with  prep/kataev/белеет_парус_9.txt Done
work with  prep/kataev/белеет_парус_10.txt Done
work with  prep/kataev/белеет_парус_11.txt Done
work with  prep/kataev/белеет_парус_12.txt Done
work with  prep/kataev/белеет_парус_13.txt Done
work with  prep/kataev/белеет_парус_14.txt Done
work with  prep/kataev/белеет_парус_15.txt Done
work with  prep/kataev/белеет_парус_16.txt Done
work with  prep/kataev/белеет_парус_17.txt Done
work with  prep/kataev/белеет_парус_18.txt Done
work with  prep/kataev/белеет_парус_19.txt Done
work with  prep/kataev/белеет_парус_20.txt Done
wo

In [127]:
#растратчики
import glob
lst_r=glob.glob('prep/kataev/Расстратчики_*')
for l in lst_r:
    with open(l, 'r') as flr:
        text=flr.read()
        with open(l.replace('.txt', ''), 'w') as flw:
            flw.write(text_clear(text, TypeOfClear=2))
            print('Done with ', l)

Done with  prep/kataev/Расстратчики_12.txt
Done with  prep/kataev/Расстратчики_11
Done with  prep/kataev/Расстратчики_6
Done with  prep/kataev/Расстратчики_5
Done with  prep/kataev/Расстратчики_2
Done with  prep/kataev/Расстратчики_3
Done with  prep/kataev/Расстратчики_9
Done with  prep/kataev/Расстратчики_1
Done with  prep/kataev/Расстратчики_4
Done with  prep/kataev/Расстратчики_10
Done with  prep/kataev/Расстратчики_7
Done with  prep/kataev/Расстратчики_8


## Шолохов: доработка

In [39]:
strROD=r'Они сражались за Родину.txt'
strROD_t=r'Они сражались за Родину-{0}.txt'
with open(auth_load.loc['sho', 'strFileDir'] +'/' + strROD, 'r') as fl:
    text=fl.read()
    
    text=re.sub('\n  ', '<BR>', text)
    text=re.split('\n<BR>', text)
    text=list(map(text_clear, text))

    for n, t in enumerate(text):
        with open(auth_load.loc['sho', 'strFileDir'] +'/' + strROD_t.format(n), 'w') as fw:
            fw.write(t)

print('all done')

all done


In [29]:
import glob

lst_kru=glob.glob('Tolstoy/*')
for i in lst_kru:

    try:
        with open(i, 'r') as fl:
            text=fl.read()
            text=re.sub(r'(?m)\n+', '\n', text)
            text=re.sub(r'(?m) +', ' ', text)
            text=re.sub(r'(?m)—', '-', text)
        
            text=re.sub(r'(?m)\*', '', text)
        with open(i, 'w') as flw:
            flw.write(text)
        print('done for {} - {}'.format(i, i))
    except IsADirectoryError:
        continue
    

Tolstoy/72
done for Tolstoy/72 - Tolstoy/72
Tolstoy/42
done for Tolstoy/42 - Tolstoy/42
Tolstoy/5
done for Tolstoy/5 - Tolstoy/5
Tolstoy/6
done for Tolstoy/6 - Tolstoy/6
Tolstoy/14
done for Tolstoy/14 - Tolstoy/14
Tolstoy/94
done for Tolstoy/94 - Tolstoy/94
Tolstoy/160
done for Tolstoy/160 - Tolstoy/160
Tolstoy/27
done for Tolstoy/27 - Tolstoy/27
Tolstoy/123
done for Tolstoy/123 - Tolstoy/123
Tolstoy/82
done for Tolstoy/82 - Tolstoy/82
Tolstoy/177
done for Tolstoy/177 - Tolstoy/177
Tolstoy/193
done for Tolstoy/193 - Tolstoy/193
Tolstoy/235
done for Tolstoy/235 - Tolstoy/235
Tolstoy/145
done for Tolstoy/145 - Tolstoy/145
Tolstoy/216
done for Tolstoy/216 - Tolstoy/216
Tolstoy/143
done for Tolstoy/143 - Tolstoy/143
Tolstoy/121
done for Tolstoy/121 - Tolstoy/121
Tolstoy/161
done for Tolstoy/161 - Tolstoy/161
Tolstoy/34
done for Tolstoy/34 - Tolstoy/34
Tolstoy/75
done for Tolstoy/75 - Tolstoy/75
Tolstoy/20
done for Tolstoy/20 - Tolstoy/20
Tolstoy/62
done for Tolstoy/62 - Tolstoy/62
Tolstoy/

done for Tolstoy/156 - Tolstoy/156
Tolstoy/234
done for Tolstoy/234 - Tolstoy/234
Tolstoy/124
done for Tolstoy/124 - Tolstoy/124
Tolstoy/144
done for Tolstoy/144 - Tolstoy/144
Tolstoy/224
done for Tolstoy/224 - Tolstoy/224
Tolstoy/102
done for Tolstoy/102 - Tolstoy/102
Tolstoy/30
done for Tolstoy/30 - Tolstoy/30
Tolstoy/96
done for Tolstoy/96 - Tolstoy/96
Tolstoy/122
done for Tolstoy/122 - Tolstoy/122
Tolstoy/97
done for Tolstoy/97 - Tolstoy/97
Tolstoy/90
done for Tolstoy/90 - Tolstoy/90
Tolstoy/211
done for Tolstoy/211 - Tolstoy/211
Tolstoy/127
done for Tolstoy/127 - Tolstoy/127
Tolstoy/204
done for Tolstoy/204 - Tolstoy/204
Tolstoy/129
done for Tolstoy/129 - Tolstoy/129
Tolstoy/183
done for Tolstoy/183 - Tolstoy/183
Tolstoy/140
done for Tolstoy/140 - Tolstoy/140
Tolstoy/136
done for Tolstoy/136 - Tolstoy/136
Tolstoy/69
done for Tolstoy/69 - Tolstoy/69
Tolstoy/130
done for Tolstoy/130 - Tolstoy/130
Tolstoy/126
done for Tolstoy/126 - Tolstoy/126
Tolstoy/53
done for Tolstoy/53 - Tolstoy